In [ ]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

role = get_execution_role()

# Specify the VPC configuration
vpc_config = {
    'SecurityGroupIds': [''],
    'Subnets': ['']
}

In [ ]:
from time import gmtime, strftime

model_name = 'sagemaker-BYOC-LR'
# model_url = 's3://{}/spacy/'.format(s3_bucket) ## MODEL S3 URL
algorithm_name = 'docker-cicd'
tag = 'v1'
container = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account_id, region, algorithm_name, tag)
instance_type = 'ml.m5.large'

print('Model name: ' + model_name)
# print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container],
    VpcConfig=vpc_config)

print("Model Arn: " + create_model_response['ModelArn'])

In [ ]:
endpoint_config_name = 'sagemaker-BYOC-LR-cf'
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

In [ ]:
%%time

import time

endpoint_name = 'sagemaker-BYOC-LR-ep'
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

In [ ]:
%%time
import json
content_type = "application/json"
request_body = {"feature": [5, 10, 15]}
#Serialize data for endpoint
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

#Endpoint invocation
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=payload)

print(response)
response_text = response['Body'].read().decode('utf-8')
print(response_text)